# Cross-encoder re-ranking

In [ ]:
import  numpy as np
from helper_utils import load_chroma, word_wrap
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from sentence_transformers import CrossEncoder

In [ ]:
embedding_function = SentenceTransformerEmbeddingFunction()

chroma_collection = load_chroma(filename='./data/microsoft_annual_report_2022.pdf', collection_name='microsoft_annual_report_2022',embedding_function=embedding_function)
chroma_collection.count()

## Re-ranking the long tail

In [ ]:
query = "What has been the investment in research and development?"
results = chroma_collection.query(query_texts=query,n_results=10,include=['documents', 'embeddings'])
retrieved_documents = results['documents'][0]

for document in results['documents'][0]:
    print(word_wrap(document))
    print('')

In [ ]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [ ]:
pairs = [[query, doc] for doc in retrieved_documents]
scores = cross_encoder.predict(pairs)
print("Scores:")
for score in scores:
    print(score)

In [ ]:
print("New Ordering:")
for o in np.argsort(scores)[::-1]:
    print(o+1)

## Re-ranking with Query Expansion

In [ ]:
original_query = "What were the most important factors that contributed to increases in revenue?"
generated_queries = [
    "What were the major drivers of revenue growth?",
    "Were there any new product launches that contributed to the increase in revenue?",
    "Did any changes in pricing or promotions impact the revenue growth?",
    "What were the key market trends that facilitated the increase in revenue?",
    "Did any acquisitions or partnerships contribute to the revenue growth?"
]

In [ ]:
queries = [original_query] + generated_queries

results = chroma_collection.query(query_texts=queries, n_results=10, include=['documents', 'embeddings'])
retrieved_documents = results['documents']

In [ ]:
# Deduplicate the retrieved documents
unique_documents = set()
for documents in retrieved_documents:
    for document in documents:
        unique_documents.add(document)

unique_documents = list(unique_documents)

In [ ]:
pairs = []
for doc in unique_documents:
    pairs.append([original_query, doc])

In [ ]:
scores = cross_encoder.predict(pairs)


In [ ]:
print("Scores:")
for score in scores:
    print(score)

In [ ]:
print("New Ordering:")
for o in np.argsort(scores)[::-1]:
    print(o)